In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format',lambda x : '%.0f' % x)
import re
import datetime

In [ ]:
df = pd.read_clipboard()

In [ ]:
df = df[(df["Group"] == "MT-SYSTEM") & (df["類型"] == "天線")]

In [ ]:
df1 = df.groupby(["Group", "預交年份", "預交月份", "品名"])[["數量"]].sum().unstack("預交月份").reset_index()

In [ ]:
df1.to_excel(r"C:\Users\kaihsu\Desktop\MT-SYSTEM_RF_Forecast_2021.xlsx")

In [ ]:
df1

In [ ]:
today = datetime.date.today().strftime('%Y-%m-%d')

In [ ]:
today

## Teltonika數據轉置

In [ ]:
re.search('([A-Za-z]+)', '1232(Bourns)')

In [ ]:
df = pd.read_excel(r"C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_Teltonika_response.xlsx", None)
df = df['2021 Forecast']

In [ ]:
df.columns = ['INPAQ PN', 'January', 'February', 'March', 'April', 'May',
              'June', 'July', 'August', 'September', 'October', 'November', 
              'December']

In [ ]:
df = df.groupby(['INPAQ PN']).sum().stack().reset_index()



In [ ]:
df.columns = ['INPAQ PN', 'Month', 'Qty.']

In [ ]:
df

## 2018~2020實績和2021預算合併

#### 1. 處理2018年數據

In [ ]:
customer_dict = {'Bourns': 'Bourns', 'WE': 'WE', 'Eaton': 'Eaton', 'LF': 'LF', '華新科技':'華新科技', '信昌':'信昌', 'Laird':'Laird', 'MT-SYSTEM':'MT-SYSTEM',
       'TELTONIKA':'TELTONIKA', 'WiSilica':'WiSilica', 'TE ':'TE', '東莞弘電':'東莞弘電', '藝感':'藝感', 'LF OEM':'LF OEM', 'Globalte':'Globalte',
       'Eteily':'Eteily', 'M.S.':'MS', 'Credence':'Credence', 'Zebra':'Zebra', 'Fides':'Fides', 'Nexty':'Nexty', 'MATE':'Reliance',
       'SSF':'Reliance', 'G -PLAST':'Reliance', 'JBJ':'Reliance', 'Univa Te':'Unified', 'Sancraft':'Reliance', 'FIRST EN':'Reliance',
                'AEM':'AEM', 'Bussmann':'Eaton', 'LairdUSA':'Laird', 'LF(TE)': 'LF', 'LITTEL': 'LF', 'MT-Sys.':'MT-SYSTEM', 'STEWARD':'Laird',
                'StewdCN':'Laird', 'WUHK':'WE', 'WUR':'WE', 'WUTJ':'WE','WuUSA':'WE', '匹克電子':'匹克電子', '東莞華匯':'華新科技', 'TELTONIK':'TELTONIKA',
                'MT-Syste':'MT-SYSTEM','MT-Sys':'MT-SYSTEM','泰連香港':'TE', 'LF.瑞侃':'LF OEM', '天灣電子':'LF OEM', '鎮江恆業':'LF OEM',
                'CAME':'CAME', 'Inter Ru':'MT-SYSTEM', 'EATON':'Eaton'}

In [ ]:
code = pd.read_excel(r"D:\pythonp_programming\INPAQ_Python_project\數據整理\BU_code.xlsx")

In [ ]:
x = code['Code'].tolist()
y = code['BG'].tolist()

In [ ]:
bg_code = dict(zip(x,y))
# bg_code.setdefault('TNF', 'EMC')
# bg_code.setdefault('MLCC', 'EMC')
# bg_code.setdefault('TNF ', 'EMC')

In [ ]:
sales2018 = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\訂單出貨明細_axmr4301_2018.xlsx")

In [ ]:
sales2018 = sales2018[sales2018['負責業務'].isin(['鄭里緗', '許凱智', '楊婉芬'])]

In [ ]:
sales2018["預交日期"] = pd.to_datetime(sales2018["預交日期"], yearfirst=True)

In [ ]:
sales2018['預交月份'] = sales2018['預交日期'].dt.month_name()

In [ ]:
sales2018['Group'] = sales2018['客戶名稱'].map(customer_dict)

In [ ]:
sales2018['負責業務'] = sales2018['負責業務'].map(lambda x: '許凱智' if x == '楊婉芬' else x)

In [ ]:
sales2018['數量'] = sales2018.apply(lambda x: x['數量'] * 1000 if x['單位'] == 'KPCS' else x['數量'], axis=1)

In [ ]:
sales2018['Category'] = sales2018['產品分類'].str[0:4]

In [ ]:
sales2018['BG'] = sales2018['Category'].map(bg_code)

In [ ]:
sales2018['類型'] = sales2018.apply(lambda x: '天線' if x['BG'] == 'RF' else '其他' if x['BG'] == 'Others' else '元件', axis=1)

In [ ]:
sales2018['預交年份'] = sales2018['預交日期'].dt.year

In [ ]:
result2018 = sales2018[['負責業務', 'BG', 'Group', '預交年份', '預交月份', '數量', '金額(NTD)', '品名', '類型']]

In [ ]:
result2018.columns = ['負責業務', 'BG', 'Group', '預交年份', '預交月份', '數量', '本國幣別NTD', '品名', '類型']

#### 2. 處理2019、2020和2021預算 20210107

In [ ]:
sales = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\歷史數據\2018_2020.xlsx", sheet_name="Sheet1")
budget = pd.read_excel(r"C:\Users\kaihsu\Desktop\2021 Forecast\Susan要的\最終版本\20210128_預算彙總表2021 RF1匯整20201230-OEM_revised_version.xlsx", None)

In [ ]:
rf_qty_budget = budget['天線數量']
rf_ntd_budget = budget['天線金額']
com_qty_budget = budget['元件數量']
com_ntd_budget = budget['元件金額']

In [ ]:
rf_qty_budget = rf_qty_budget.groupby(['業務名稱', 'Group', 'BG', '品名']).sum().stack().reset_index()
rf_qty_budget.columns = ['負責業務', 'Group', 'BG', '品名', '預交月份', '數量']
rf_ntd_budget = rf_ntd_budget.groupby(['業務名稱', 'Group', 'BG', '品名']).sum().stack().reset_index()
rf_ntd_budget.columns = ['負責業務', 'Group', 'BG', '品名', '預交月份', '本國幣別NTD']

In [ ]:
total = rf_qty_budget.merge(rf_ntd_budget, on=['負責業務', 'Group', 'BG', '品名', '預交月份'], how='left')

In [ ]:
com_qty_budget = com_qty_budget.groupby(['業務名稱', 'Group', 'BG', '品名']).sum().stack().reset_index()
com_qty_budget.columns = ['負責業務', 'Group', 'BG', '品名', '預交月份', '數量']
com_ntd_budget = com_ntd_budget.groupby(['業務名稱', 'Group', 'BG', '品名']).sum().stack().reset_index()
com_ntd_budget.columns = ['負責業務', 'Group', 'BG', '品名', '預交月份', '本國幣別NTD']
com_total = com_qty_budget.merge(com_ntd_budget, on=['負責業務', 'Group', 'BG', '品名', '預交月份'], how='left')

In [ ]:
final_budget = pd.concat([total, com_total], axis=0)

In [ ]:
month_dict = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
             7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
final_budget['預交月份'] = final_budget['預交月份'].map(month_dict)
# sales['預交月份'] = sales['預交月份'].map(month_dict)

In [ ]:
final_budget = final_budget.reindex(columns=['負責業務', 'BG', 'Group', '預交月份', '數量', '本國幣別NTD', '品名'])

In [ ]:
final_budget.insert(3, '預交年份', 2021)

In [ ]:
final_budget['類型'] = final_budget.apply(lambda x: '天線' if x['BG'] == 'RF' else '元件', axis=1)

In [ ]:
sales['產品code'] = sales['產品分類'].str[:4]

In [ ]:
sales['BG'] = sales['產品code'].map(bg_code)

In [ ]:
sales = sales[(sales['狀態'] != '合約') & (sales['狀態'] != '待通知')][['負責業務', 'BG', '客戶名稱', '年份', '預交月份', '數量', '金額*集團匯率(NTD)', '品名']]

In [ ]:
x = pd.read_clipboard()

In [ ]:
cus_key = x['Cus'].tolist()
cus_value = x['Group'].tolist()
cus_dict = dict(zip(cus_key, cus_value))

In [ ]:
sales['Group'] = sales['客戶名稱'].map(cus_dict)

In [ ]:
sales['類型'] = sales.apply(lambda x: '天線' if x['BG'] == 'RF' else '元件' if x['Group'] == 'LF OEM' else '其他' if x['BG'] == 'Others' else '元件', axis=1)

In [ ]:
sales = sales[['負責業務', 'BG', 'Group', '年份', '預交月份', '數量', '金額*集團匯率(NTD)', '品名', '類型']]

In [ ]:
sales.columns = ['負責業務', 'BG', 'Group', '預交年份', '預交月份', '數量', '本國幣別NTD', '品名', '類型']

In [ ]:
mohan_customer = {'Eaton':'Eaton', 'TELTONIKA':'TELTONIKA', '信昌':'信昌', '華新科技':'華新科技',
                  'Laird':'Laird', 'WE':'WE', 'Bourns':'Bourns', 'LF':'LF',
       'MT-SYSTEM':'MT-SYSTEM', 'WiSilica':'WiSilica', 'TE':'TE', 'LF OEM':'LF OEM', '藝感':'藝感',
                  'Globalte':'Globalte', 'Eteily':'Eteily', 'Credence':'Credence', 'M.S.':'MS',
                  'Zebra':'Zebra', 'Fides':'Fides', 'Nexty':'Nexty', 'SSF':'Reliance',
       'JBJ': 'Reliance', 'UnivaTe':'Unified', 'G-PLAST':'Reliance', 'G -PLAST': 'Reliance', 'FIRST EN':'Reliance',
                  'MATE':'Reliance', 'Sancraft':'Reliance', 'Univa Te': 'Unified', '同方電子': 'LF OEM'}

In [ ]:
final_budget['Group'] = final_budget.apply(lambda x: '華新科技' if x['Group'] == '東莞華匯' else x['Group'], axis=1)

In [ ]:
final_budget['Group'] = final_budget['Group'].apply(lambda x: 'TELTONIKA' if x == 'Teltonika' else 'MT-SYSTEM' if x == 'MT-System' else x)

In [ ]:
final_budget['負責業務'] = final_budget['負責業務'].apply(lambda x: '墨漢雷迪' if x == '墨漢' else '鄭里緗' if x == '沈思明' else x)

In [ ]:
result = pd.concat([sales, final_budget], axis=0)

In [ ]:
result['BG'] = result.apply(lambda x: 'ODM' if x['Group'] == 'LF OEM' else x['BG'], axis=1)

In [ ]:
result['數量'] = result['數量'].astype(int)
result['本國幣別NTD'] = result['本國幣別NTD'].astype(int)

In [ ]:
result["BG"].unique()

In [ ]:
result.to_excel(r'C:\Users\kaihsu\Desktop\業績總表\最終預算報告用表格（2021年初Susan要求修改版）.xlsx', index=False)

In [ ]:
import plotly.express as px

In [ ]:
result.columns

In [ ]:
fig1 = result.groupby(['預交年份', '預交月份', '類型'])[['本國幣別NTD']].sum().reset_index()

In [ ]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
               'August', 'September', 'October', 'November', 'December']
fig1['預交月份'] =pd.Categorical(fig1['預交月份'], ordered=True, categories=month_order)
fig1 = fig1.sort_values(by='預交月份')

In [ ]:
result.columns

In [ ]:
bar_plot = result.groupby(['預交年份', '負責業務', 'Group', '預交月份', 'BG'])[['數量', '本國幣別NTD']].sum().reset_index()

In [ ]:
fig2 = px.histogram(bar_plot.query('類型 == "天線"'), x='Group', y='本國幣別NTD', color='預交年份', barmode='group', labels={'本國幣別NTD':'營業額', 'Group':'Customer', '預交年份':'Year'}, title='天線客戶歷年營收').update_xaxes(categoryorder='total descending').show()

In [ ]:
pie_data = result[result['預交年份'] == 2020]
pie_group = pie_data.groupby(['負責業務'])[['本國幣別NTD']].sum().reset_index()
px.pie(pie_group, values='本國幣別NTD', names='負責業務')


In [ ]:
px.treemap(bar_plot, path=['預交年份', 'BG', 'Group'], values='本國幣別NTD', color='數量')

## 預算整理對外版本

In [ ]:
for_cp = result[(result['預交年份'].isin([2020,2021])) & (result['負責業務'] == '許凱智') & (result['Group'] == '華新科技') & (result['類型'] == '元件')]

In [ ]:
sort_product = for_cp['品名'].str.extract('([A-Z]+)(\d{,4})', expand=True)
sort_product.columns = ['產品別', '尺寸']

In [ ]:
for_cp = pd.concat([for_cp, sort_product], axis=1)

In [ ]:
for_tree = for_cp.groupby(['預交年份', '預交月份','BG', '產品別', '尺寸', '品名'])[['本國幣別NTD', '數量']].sum().reset_index()

In [ ]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
               'August', 'September', 'October', 'November', 'December']
for_tree['預交月份'] = pd.Categorical(for_tree['預交月份'], ordered=True, categories=month_order)
for_tree = for_tree.sort_values(by='預交月份')

In [ ]:
import plotly.express as px

In [ ]:
for_tree = for_tree[for_tree['本國幣別NTD'] != 0]

In [ ]:
fig = px.treemap(for_tree, title='華新科技產品樹狀圖',
                 path =['預交年份', '預交月份', '產品別', '品名'], 
                values='本國幣別NTD', 
                color='數量')

fig.show()

In [ ]:
table = for_tree.groupby(['預交年份', 'BG', '產品別'])[['本國幣別NTD', '數量']].sum().reset_index()
table.columns = ['年份', 'BU', '產品別', '金額', '數量']

In [ ]:
table = table.sort_values(by=['年份','金額'], ascending=False)

In [ ]:
format_dict = {'金額': '{0:,.0f}', '數量': '{0:,.0f}'}
table = table.style.format(format_dict).set_caption('各BU產品別排名').hide_index()

In [ ]:
table

In [ ]:
his_fig = px.histogram(for_tree, x='預交月份', y='本國幣別NTD', color='預交年份', barmode='group',
            title='2020/2021實績 vs 預算',
            labels={'預交月份': '月份',
                   '本國幣別NTD': '金額',
                   '預交年份': '年份'})

his_fig.show()

In [ ]:
with open('華新科預算修正.html', 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(his_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(table.render())
    f.close()

In [ ]:
final_budget[(final_budget['Group'].isin(['信昌', '華新科技', 'TELTONIKA'])) & (final_budget['類型'] == '元件')].groupby(['Group'])[['數量', '本國幣別NTD']].sum().reset_index()

## 2021 Budget test

* 如果後續要groupby，需要groupby的欄位不可以為空值

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_final.xlsx', sheet_name='2021預算總表')


In [ ]:
df2 = df[df['Unit'] == 'KPCS']
df2['Qty.'] = df2['Qty.'].apply(lambda x: x*1000)
# df2
# df3 = df[df['Unit'] == 'PCS']

In [ ]:
df3 = df[df['Unit'] == 'PCS']

In [ ]:
df2['Unit'] = df2['Unit'].str.replace('KPCS', 'PCS')
# df2

In [ ]:
df_final = pd.concat([df2, df3], axis=0)

In [ ]:
# df_final = df[df_final['Year'] == 2021].reset_index(drop=True)
df_final['Application'] = df_final['Application'].fillna('Unknown')
df_final['End customer'] = df_final['End customer'].fillna('Unknown')
df_final

In [ ]:
test = df_final.groupby(['Distributor', 'End customer', 'Product category', 'INPAQ PN', 'Application', 'Unit Price', 'Currency', 'Month'])[['Qty.']].sum().reset_index()

In [ ]:
# 重新改變columns的排列
test = test.reindex(columns=['Distributor', 'End customer', 'Product category', 'INPAQ PN', 'Application', 'Qty.', 'Unit Price', 'Currency', 'Month', 'Total NTD'])
# test.rename(columns={'Unit Price': 'Unit Price_USD/RMB'})

In [ ]:
test = pd.pivot_table(test, values='Qty.', index=['Distributor', 'End customer', 'Product category', 'INPAQ PN', 'Application', 'Unit Price', 'Currency'], columns='Month').reset_index()

In [ ]:
test

In [ ]:
test.to_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_by_Susan_format.xlsx', sheet_name='Forecast_yearly', index=False)

### 20201217修改Teltonika預算

In [ ]:
df = df[['Distributor', 'End customer', 'Product category', 'Application', 'INPAQ PN', 'Currency', 'Unit Price', 'Qty.', 'Month']]
df = df[(df['Distributor'] == 'Teltonika') & (df['Product category'] == 'RF')]

In [ ]:
df = pd.pivot_table(df, values='Qty.', index=['Distributor', 'End customer', 'Product category', 'Application', 'INPAQ PN', 'Currency', 'Unit Price'], columns='Month').reset_index()

In [ ]:
df.to_excel(r'C:\Users\kaihsu\Desktop\2021_forecast_teltonika_revised_20201217.xlsx', index=False)

## 整理藝感的年度預估


In [ ]:
df = pd.read_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_藝感_response.xlsx', None)

In [ ]:
df = df['2021 Forecast']

In [ ]:
df = df.drop([0,33])

In [ ]:
df = df.drop('Customer PN', axis=1)


In [ ]:
df = df.set_index('INPAQ PN')


In [ ]:
df = df.stack()


In [ ]:
df = df.reset_index()

In [ ]:
df.to_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021預算藝感整理過.xlsx')

## 整理信昌預算


In [ ]:
df = pd.read_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_PDC_response.xlsx', None)

In [ ]:
df = df['2020出貨量']

In [ ]:
df = df.drop([0,125])

In [ ]:
df = df.drop(['Unnamed: 0', 'Unnamed: 3', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], axis=1)

In [ ]:
columns=['INPAQ PN', 'Customer PN', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
df.columns = columns

In [ ]:
df = df.set_index(['INPAQ PN', 'Customer PN'])

In [ ]:
df = df.stack()

In [ ]:
df = df.reset_index()

In [ ]:
columns = ['INPAQ PN', 'Customer PN', 'Month', 'Qty.']
df.columns = columns
df

In [ ]:
df = df.reindex(columns=df.columns.tolist() + new_columns)

In [ ]:
re_columns = ['End customer', 'Distributor', 'Year', 'Month', 'Product category', 'Sub-category', 'Customer PN', 'INPAQ PN', 'Application', 'Qty.', 'Unit', 'Unit Price', 'Currency', 'Rate', 'NTD', 'Total NTD']
df = df.reindex(columns=re_columns)


In [ ]:
df['End customer'] = df['End customer'].fillna('N/A')
df['Distributor'] = df['Distributor'].fillna('信昌')
df['Year'] = df['Year'].fillna(2021)
df['Application'] = df['Application'].fillna('N/A')
df['Unit'] = df['Unit'].fillna('PCS')


In [ ]:
df['Year'] = df['Year'].astype('int')
df.to_excel(r'C:\Users\kaihsu\Desktop\2021 Forecast\2021_forecast_PDC_清洗完.xlsx', index=False)